# 🍷 Notebook de Scraping y Análisis de Vinos Vivino

Este notebook está preparado para iniciar el scraping, análisis y visualización de datos de vinos extraídos de Vivino. Sigue el esquema recomendado para cada sección clave del proceso.

**Proyecto:** Wine-IA-Web  
**Fecha de creación:** 16 de julio de 2025  
**Objetivo:** Extraer datos de vinos de Vivino.com para análisis y almacenamiento  
**Lenguaje:** Python 3.12  
**Frameworks:** Selenium, BeautifulSoup, Flask

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import csv
import os
from datetime import datetime
from flask import Flask, render_template, request, jsonify

## Configurar entorno virtual y dependencias

Para asegurar un entorno limpio y reproducible, crea un entorno virtual y instala las dependencias necesarias:

```bash
python -m venv wine_env
wine_env\Scripts\activate
pip install selenium beautifulsoup4 flask webdriver-manager
```

Puedes añadir estas líneas en una terminal antes de ejecutar el notebook.

In [7]:
# Inicializar Selenium WebDriver con WebDriver Manager
chrome_options = Options()
chrome_options.add_argument('--headless')              # Sin interfaz gráfica
chrome_options.add_argument('--disable-gpu')           # Deshabilitar GPU
chrome_options.add_argument('--no-sandbox')            # Sin sandbox
chrome_options.add_argument('--window-size=1920,1080') # Resolución fija
chrome_options.add_argument('--disable-dev-shm-usage') # Optimización memoria
chrome_options.add_argument('--disable-web-security')  # Bypass seguridad web
chrome_options.add_argument('--disable-features=VizDisplayCompositor')
chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)


In [8]:
# Acceder a la URL objetivo y cargar la página de GuiaPenin
url_guia = "https://guiapenin.wine/vinos/?moneda=EUR&rating_min=3.8&order=best-picks&price_min=7&price_max=60&region=España&type=tinto,garnacha"
driver.get(url_guia)
time.sleep(5)  # Esperar a que la página cargue completamente
html = driver.page_source
print("Página de GuiaPenin cargada correctamente.")


Página de GuiaPenin cargada correctamente.


In [9]:
# Extraer tarjetas y enlaces de vinos usando BeautifulSoup (GuiaPenin)
soup = BeautifulSoup(html, 'html.parser')

# Buscar tarjetas de vino en GuiaPenin
wine_cards = soup.find_all('div', class_='wine-card')
print(f"Tarjetas de vino encontradas: {len(wine_cards)}")

# Buscar enlaces de vinos en GuiaPenin
wine_links = soup.find_all('a', href=lambda x: x and '/vino/' in str(x))
print(f"Enlaces de vinos encontrados: {len(wine_links)}")


Tarjetas de vino encontradas: 0
Enlaces de vinos encontrados: 0


In [10]:
# Extraer precios, nombres y URLs de cada vino (GuiaPenin)
import re
vinos_extraidos = []
for idx, card in enumerate(wine_cards):
    # Nombre del vino
    nombre_tag = card.find('h3', class_='wine-title')
    nombre = nombre_tag.get_text(strip=True) if nombre_tag else card.get_text(separator=" ", strip=True)
    # Buscar precio
    precio_tag = card.find('span', class_='wine-price')
    precio_eur = None
    if precio_tag:
        precio_match = re.search(r'(\d+[,.]?\d*)\s*€', precio_tag.get_text())
        if precio_match:
            precio_eur = float(precio_match.group(1).replace(',', '.'))
    # Buscar URL
    url = None
    link = card.find('a', href=True)
    if link and '/vino/' in link['href']:
        url = link['href']
        if not url.startswith('http'):
            url = 'https://guiapenin.wine' + url
    vinos_extraidos.append({
        'nombre': nombre,
        'precio_eur': precio_eur,
        'url': url,
        'posicion': idx + 1
    })
print(f"Vinos extraídos: {len(vinos_extraidos)}")
print(vinos_extraidos[:3])  # Mostrar los primeros 3 vinos extraídos


Vinos extraídos: 0
[]


In [ ]:
# Guardar datos en archivos CSV (GuiaPenin)
import pandas as pd

def guardar_datos_csv(vinos, nombre_archivo):
    df = pd.DataFrame(vinos)
    df.to_csv(nombre_archivo, index=False, encoding='utf-8')
    print(f"Datos guardados en {nombre_archivo}")

# Guardar archivo individual por sesión
fecha = datetime.now().strftime('%Y%m%d_%H%M%S')
nombre_individual = f"datos_scraping/guia_scraping_{fecha}.csv"
guardar_datos_csv(vinos_extraidos, nombre_individual)

# Guardar en archivo histórico consolidado
nombre_historico = "datos_scraping/guia_historico.csv"
if os.path.exists(nombre_historico):
    df_hist = pd.read_csv(nombre_historico)
    df_nuevo = pd.DataFrame(vinos_extraidos)
    df_hist = pd.concat([df_hist, df_nuevo], ignore_index=True)
    df_hist.to_csv(nombre_historico, index=False, encoding='utf-8')
    print(f"Datos agregados al histórico {nombre_historico}")
else:
    guardar_datos_csv(vinos_extraidos, nombre_historico)


In [ ]:
# Visualización de los resultados extraídos
import pandas as pd
from IPython.display import display

df_vinos = pd.DataFrame(vinos_extraidos)
display(df_vinos.head(10))  # Mostrar los primeros 10 vinos

# Estadísticas básicas
print(f"Total de vinos extraídos: {len(df_vinos)}")
if 'precio_eur' in df_vinos.columns:
    print(f"Precio promedio: {df_vinos['precio_eur'].mean():.2f} €")
    print(f"Precio mínimo: {df_vinos['precio_eur'].min():.2f} €")
    print(f"Precio máximo: {df_vinos['precio_eur'].max():.2f} €")


In [11]:
# Extraer datos de una página de detalle de vino en GuiaPenin
# Ejemplo para https://guiapenin.wine/guide/wine/althay_2023_rd/es

url_detalle = "https://guiapenin.wine/guide/wine/althay_2023_rd/es"
driver.get(url_detalle)
time.sleep(3)
html_detalle = driver.page_source
soup_detalle = BeautifulSoup(html_detalle, 'html.parser')

# Nombre del vino
nombre = soup_detalle.find('h1').get_text(strip=True) if soup_detalle.find('h1') else None
# Puntuación (puede estar en un span o div con clase específica)
puntuacion = None
punt_tag = soup_detalle.find('span', class_='wine-score')
if punt_tag:
    puntuacion = punt_tag.get_text(strip=True)
# Precio (si está disponible)
precio = None
precio_tag = soup_detalle.find('span', class_='wine-price')
if precio_tag:
    precio = precio_tag.get_text(strip=True)
# Bodega
bodega = None
bodega_tag = soup_detalle.find('div', class_='wine-producer')
if bodega_tag:
    bodega = bodega_tag.get_text(strip=True)

print(f"Nombre: {nombre}")
print(f"Puntuación: {puntuacion}")
print(f"Precio: {precio}")
print(f"Bodega: {bodega}")


Nombre: Althay 2023
Puntuación: None
Precio: None
Bodega: None
